In [ ]:
import numpy as np, pandas as pd
load = ['HasDetections', 'AvSigVersion', 'Census_OSVersion', 'OsBuildLab']
df_train = pd.read_csv('../input/microsoft-malware-prediction/train.csv',dtype='category',usecols=load)
df_train['HasDetections'] = df_train['HasDetections'].astype('int8')


In [ ]:
from datetime import datetime, date, timedelta

# AS timestamp
datedictAS = np.load('../input/malware-timestamps/AvSigVersionTimestamps.npy',allow_pickle=True)[()]
df_train['DateAS'] = df_train['AvSigVersion'].map(datedictAS)  

# OS timestamp
datedictOS = np.load('../input/malware-timestamps-2/OSVersionTimestamps.npy',allow_pickle=True)[()]
df_train['DateOS'] = df_train['Census_OSVersion'].map(datedictOS)  

# BL timestamp
def convert(x):
    try:
        d = datetime.strptime(x.split('.')[4],'%y%m%d-%H%M')
    except:
        d = np.nan
    return d
df_train['DateBL'] = df_train['OsBuildLab'].map(convert)
df_train.head()

In [ ]:
data = pd.read_csv('../input/google-safe-browsing-transparency-report-data/data.csv')
data['WeekOf'] = data['WeekOf'].map(lambda x: datetime.strptime(x,'%Y-%m-%d').date())
datedictAS = np.load('../input/malware-timestamps/AvSigVersionTimestamps.npy',allow_pickle=True)[()]
weekdictAS={}
for x in datedictAS: 
    weekdictAS[x] = (datedictAS[x] - timedelta(days= -7+1+datedictAS[x].weekday())).date()
df_train['WeekOf'] = df_train['AvSigVersion'].map(weekdictAS)
df_train = pd.merge(df_train, data, on='WeekOf', how='left')
print('GOOGLE DATA')
data.sample(5)

In [ ]:
data2 = pd.read_csv('../input/malware-avsigversion-threats/AvSigversion_Threats.csv')
cv = pd.DataFrame(data2.groupby('AvSigVersion')['index'].count()).rename({'index':'ThreatCount'},axis=1)
df_train = pd.merge(df_train,cv,on='AvSigVersion',how='left')
df_train['ThreatCount'].fillna(0,inplace=True)
print('THREAT DATA')
data2.sample(10)

In [ ]:
del df_train['DateAS'], df_train['DateOS'], df_train['DateBL'], df_train['WeekOf'] 
del df_train['AvSigVersion'], df_train['OsBuildLab'], df_train['Census_OSVersion']
print('TRAIN DATA')
df_train.sample(5)

In [ ]:
import numpy as np, pandas as pd
load = ['HasDetections', 'AvSigVersion', 'Census_OSVersion', 'OsBuildLab']
df_test = pd.read_csv('../input/microsoft-malware-prediction/train.csv',dtype='category',usecols=load)
df_test['HasDetections'] = df_test['HasDetections'].astype('int8')


In [ ]:
from datetime import datetime, date, timedelta

# AS timestamp
datedictAS = np.load('../input/malware-timestamps/AvSigVersionTimestamps.npy',allow_pickle=True)[()]
df_test['DateAS'] = df_test['AvSigVersion'].map(datedictAS)  

# OS timestamp
datedictOS = np.load('../input/malware-timestamps-2/OSVersionTimestamps.npy',allow_pickle=True)[()]
df_test['DateOS'] = df_test['Census_OSVersion'].map(datedictOS)  

# BL timestamp
def convert(x):
    try:
        d = datetime.strptime(x.split('.')[4],'%y%m%d-%H%M')
    except:
        d = np.nan
    return d
df_test['DateBL'] = df_test['OsBuildLab'].map(convert)
df_test.head()

In [ ]:
data = pd.read_csv('../input/google-safe-browsing-transparency-report-data/data.csv')
data['WeekOf'] = data['WeekOf'].map(lambda x: datetime.strptime(x,'%Y-%m-%d').date())
datedictAS = np.load('../input/malware-timestamps/AvSigVersionTimestamps.npy',allow_pickle=True)[()]
weekdictAS={}
for x in datedictAS: 
    weekdictAS[x] = (datedictAS[x] - timedelta(days= -7+1+datedictAS[x].weekday())).date()
df_test['WeekOf'] = df_test['AvSigVersion'].map(weekdictAS)
df_test = pd.merge(df_test, data, on='WeekOf', how='left')
print('GOOGLE DATA')
data.sample(5)

In [ ]:
data2 = pd.read_csv('../input/malware-avsigversion-threats/AvSigversion_Threats.csv')
cv = pd.DataFrame(data2.groupby('AvSigVersion')['index'].count()).rename({'index':'ThreatCount'},axis=1)
df_test = pd.merge(df_test,cv,on='AvSigVersion',how='left')
df_test['ThreatCount'].fillna(0,inplace=True)
print('THREAT DATA')
data2.sample(10)

In [ ]:
del df_test['DateAS'], df_test['DateOS'], df_test['DateBL'], df_test['WeekOf'] 
del df_test['AvSigVersion'], df_test['OsBuildLab'], df_test['Census_OSVersion']
print('TEST DATA')
df_test.sample(5)

In [ ]:
print(len(df_test))

In [ ]:
#PREPROCESSING

#1) DIMENTIONALITY REDUCTION TEST DATA

#based on missing value ratio
test_df_orig= df_test
filtered_col = df_test.columns[df_test.isnull().mean()>0.5]
print(filtered_col)
print(len(df_test.columns))
df_test=df_test.drop(list(filtered_col),axis=1)
print(len(df_test.columns))
#test_df=test_df.drop(list(filtered_col),axis=1)

#based on the variance of each coloumn
#An attribute with low variance not really separating data points in anyway.so we can remove that coloumn
l=[]
threshold=0.8
for i in df_test.columns:
    max_rfrequency = df_test[i].value_counts(normalize=True, dropna=False).values[0]#finding maximum value of the relative frequency of values in that column
    if(max_rfrequency>threshold):
        l.append(i)
print(l)
print(len(l))
df_test=df_test.drop(l,axis=1)
print(len(df_test.columns))
#test_df=test_df.drop(l,axis=1)

#REPLACING MISSING VALUES IN EACH ROW 
#since majority of the columns are categorical we are replacing the missing values by mode
for column in df_test.columns:
    df_test[column].fillna(df_test[column].mode()[0], inplace=True)


In [ ]:
#PREPROCESSING

#1) DIMENTIONALITY REDUCTION TRAIN DATA

#based on missing value ratio
train_df_orig= df_train
filtered_col = df_train.columns[df_train.isnull().mean()>0.5]
print(filtered_col)
print(len(df_train.columns))
df_train=df_train.drop(list(filtered_col),axis=1)
print(len(df_train.columns))
#test_df=test_df.drop(list(filtered_col),axis=1)

#based on the variance of each coloumn
#An attribute with low variance not really separating data points in anyway.so we can remove that coloumn
l=[]
threshold=0.8
for i in df_train.columns:
    max_rfrequency = df_train[i].value_counts(normalize=True, dropna=False).values[0]#finding maximum value of the relative frequency of values in that column
    if(max_rfrequency>threshold):
        l.append(i)
print(l)
print(len(l))
df_train=df_train.drop(l,axis=1)
print(len(df_train.columns))
#test_df=test_df.drop(l,axis=1)

#REPLACING MISSING VALUES IN EACH ROW 
#since majority of the columns are categorical we are replacing the missing values by mode
for column in df_train.columns:
    df_train[column].fillna(df_train[column].mode()[0], inplace=True)


In [ ]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [ ]:
train_target = df_train['HasDetections']
del df_train['HasDetections']
test_target = df_test['HasDetections']
del df_test['HasDetections']


In [ ]:
train = df_train
test = df_test
seq_len = len(df_train)
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, 4)))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=128, callbacks=[chk], validation_data=(test,test_target))
#loading the model and checking accuracy on the test data
model = load_model('best_model.pkl')

from sklearn.metrics import accuracy_score
test_preds = model.predict_classes(test)
accuracy_score(test_target, test_preds)